# 📊 **01 — Exploratory Data Analysis (EDA)**

**Notebook Purpose:**
Perform a *light, structured exploration* of the Kaggle **Detect Reversal Points in U.S. Equities** dataset. This notebook focuses on understanding distributions, spotting anomalies, mapping feature groups, and laying the groundwork for preprocessing and feature engineering.

---

**Competition:** *Detect Reversal Points in US Equities*
**Deadline:** December 31, 2025
**Repository:** `Kaggle-Detect-Reversal-Points-in-US-Equities`
**Author:** Brice Nelson

---

**Notebook Date Created:** 2025-11-29
**Notebook Last Updated:** 2025-11-29

---

## 🧭 **Goals of This Notebook**

- Load raw Kaggle training and test data (`/data/raw/`)
- Inspect dataset shape, dtypes, and missing values
- Explore Signal Descriptor columns
- Examine target class distribution (0–3)
- Identify time-related patterns and potential leakage risks
- Save EDA plots into `/figures/eda/`
- Produce initial notes for feature engineering

---

## 📂 **References**

- Project Plan: `docs/00_overview/01_reversal_points_project_plan.md`
- Folder Explanations: `docs/01_architecture/02_folder_explanations.md`
- Project Structure: `docs/01_architecture/01_project_structure.md`


In [1]:
# import libraries

import os
import pandas as pd
import numpy as np
import duckdb
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno

# configurations
sns.set_theme(style="darkgrid")
pd.set_option("display.max_columns", None)
pd.set_option("display.width", 120)

plt.style.use("seaborn-v0_8")


### Load Raw Data
- Load via Duckdb
- Create a connection
- Load the training and test datasets into dataframes


In [3]:
# Create duckdb connection

conn = duckdb.connect()

# Create duckdb dataframe

train_df = conn.execute("""
                        SELECT * FROM
                            read_csv_auto('../data/raw/new_competition_data/train.csv',
                            max_line_size=5000000)""").df()
test_df = conn.execute("""
                       SELECT * FROM
                           read_csv_auto('../data/raw/new_competition_data/test.csv',
                           max_line_size=5000000)""").df()

print('Train dataframe created.')
print('Test dataframe created.')

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Train dataframe created.
Test dataframe created.


## Explore Descriptive Statistics

In [4]:
# Shape

print('Train dataframe shape:', train_df.shape)
print('Test dataframe shape:', test_df.shape)


Train dataframe shape: (2683, 68507)
Test dataframe shape: (1151, 68506)


In [6]:
# Train df info

train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2683 entries, 0 to 2682
Columns: 68507 entries, train_id to class_label
dtypes: bool(68499), datetime64[us](1), float64(4), int64(1), object(2)
memory usage: 175.4+ MB


In [7]:
# Test df info

test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1151 entries, 0 to 1150
Columns: 68506 entries, id to zone_99.5
dtypes: bool(68499), datetime64[us](1), float64(4), int64(1), object(1)
memory usage: 75.3+ MB


## 🦆 Why Use DuckDB for This EDA?

This dataset is extremely *wide* — more than **68,000 columns** of boolean and numeric features.
Instead of forcing pandas to scan tens of thousands of columns, DuckDB lets us work directly with the dataset **metadata**, making EDA faster and more efficient.

### 🔑 Key Advantages
- **Instant schema inspection** via `pragma_table_info()`
- **Fast prefix-based grouping** to discover feature families
- **Easy train/test column alignment checks**
- **Vectorized SQL operations** that scale better than Python loops

In short:
**DuckDB handles wide datasets with ease, giving us a cleaner, faster EDA workflow.**


## 🔍 DuckDB + Pandas Data Workflow (Short Summary)

For this project, we use a hybrid approach that combines the speed of **DuckDB** with the flexibility of **pandas**:

1. **Load CSVs with DuckDB** using `read_csv_auto()` for fast ingestion of 68,000+ columns.
2. **Convert results to pandas** via `.df()` so the data is ready for modeling (LightGBM, sklearn, etc.).
3. **Register the pandas DataFrames back into DuckDB** so we can run fast SQL queries for:
   - prefix-based feature grouping
   - schema comparisons (train vs. test)
   - constant-column detection
   - metadata inspection via `pragma_table_info()`

This workflow gives us **fast schema exploration** with DuckDB and **full ML compatibility** with pandas — the best setup for extremely wide datasets.


In [12]:
# Register for SQL use
duckdb.register('train', train_df)
duckdb.register('test', test_df)

# Convert column names into pandas DataFrames
col_df = pd.DataFrame({'column_name': train_df.columns})
duckdb.register('cols', col_df)

# Extract prefixes and count them
duckdb.query("""
    SELECT
        regexp_extract(column_name, '^[^_]+') AS prefix,
        COUNT(*) AS count
    FROM cols
    GROUP BY prefix
    ORDER BY count DESC
""").df()



,prefix,count
0,occurs,34220
1,happens,34220
2,cross,26
3,zone,13
4,trending,10
5,troughs,5
6,peaks,5
7,sm,2
8,t,1
9,momentum,1
